In [1]:
# Import Libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX 
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML

import warnings
warnings.filterwarnings("ignore")


In [2]:
# Reading Dataframe
df = pd.read_csv('/Users/devonmarcus/Data Science Bootcamp/Ticketmaster Project/data/Event_Information_Dataframe_Part_4.csv', sep='\t')
df.drop("Unnamed: 0", axis=1, inplace=True)
pd.set_option('display.max_columns', 999)

In [3]:
# Making Ajustments to row and columns
df['Year'] = df['Year'].astype(str)
df['Month'] = df['Month'].astype(str).str.zfill(2)
df['Day'] = df['Day'].astype(str).str.zfill(2)
df["Date"] = df["Year"] + '-' + df["Month"] + '-' + df["Day"]
df = df[df['Home_Name'].notna()]
df = df[df['Genre_Home'].notna()]

In [4]:
# Filtering Columns

df = df.iloc[: , [0,1,53,54,5,30,32,33,34,35,27,55,26,23,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,40,41,42,43,44,45,47,48,49,50,51]]


In [5]:
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [6]:
output = widgets.Output()
plot_output = widgets.Output()
plot_output2 = widgets.Output()
plot_output3 = widgets.Output()

dropdown_Venue_Country = widgets.Dropdown(description='Country:',options = unique_sorted_values_plus_ALL(df.Venue_Country))
dropdown_Venue_City = widgets.Dropdown(description='City:',options = unique_sorted_values_plus_ALL(df.Venue_City))
dropdown_Segment_Home = widgets.Dropdown(description='Segment:',options = unique_sorted_values_plus_ALL(df.Segment_Home))
dropdown_Genre_Home = widgets.Dropdown(description='Genre:',options = unique_sorted_values_plus_ALL(df.Genre_Home))
dropdown_Home_Name = widgets.Dropdown(description='Home:',options = unique_sorted_values_plus_ALL(df.Home_Name))
bounded_date = widgets.DatePicker(description='Pick a Date',disabled=False)

def colour(value, comparison):
    
        if (str(value)[0:10]) == (str(comparison)[0:10]):
            return 'color: red'
        else:
            return 'color: black'
    
def common_filtering(Venue_Country, Venue_City, Segment_Home, Genre_Home,Home_Name, date):
    output.clear_output()
    plot_output.clear_output()
    plot_output2.clear_output()
    plot_output3.clear_output()

    if (Venue_Country == ALL) & (Venue_City == ALL) & (Segment_Home == ALL) & (Genre_Home == ALL)& (Home_Name == ALL):
        common_filter = df
        
    elif (Venue_City == ALL) & (Segment_Home == ALL) & (Genre_Home == ALL) & (Home_Name == ALL):
        common_filter = df[(df.Venue_Country == Venue_Country)]   
        
    elif (Segment_Home == ALL) & (Genre_Home == ALL) & (Home_Name == ALL):
        common_filter = df[(df.Venue_Country == Venue_Country) & (df.Venue_City == Venue_City)]
    
    elif (Genre_Home == ALL) & (Home_Name == ALL):
        common_filter = df[(df.Venue_Country == Venue_Country) & (df.Venue_City == Venue_City) & (df.Segment_Home == Segment_Home)]

    elif (Home_Name == ALL):
        common_filter = df[(df.Venue_Country == Venue_Country) & (df.Venue_City == Venue_City) & (df.Segment_Home == Segment_Home) & (df.Genre_Home == Genre_Home)]

    else:
        common_filter =  df[(df.Venue_Country == Venue_Country) & (df.Venue_City == Venue_City) & (df.Segment_Home == Segment_Home) & (df.Genre_Home == Genre_Home) & (df.Home_Name == Home_Name)]

    with output:
        display(common_filter.style.applymap(lambda x: colour(x, date),subset=['Date']))

    with plot_output:
        sns.kdeplot(common_filter['Price_Ranges_Min'], shade=True)
        plt.show()   
    with plot_output2:
        sns.kdeplot(common_filter['Price_Ranges_Avg'], shade=True)
        plt.show()     
    with plot_output3:
        sns.kdeplot(common_filter['Price_Ranges_Max'], shade=True)
        plt.show()  
        
def dropdown_Venue_Country_eventhandler(change):
    common_filtering(change.new, dropdown_Venue_City, dropdown_Segment_Home.value, dropdown_Genre_Home.value, dropdown_Home_Name.value, bounded_date.value)        
def dropdown_Venue_City_eventhandler(change):
    common_filtering(dropdown_Venue_Country.value, change.new, dropdown_Segment_Home.value, dropdown_Genre_Home.value, dropdown_Home_Name.value, bounded_date.value)
def dropdown_Segment_Home_eventhandler(change):
    common_filtering(dropdown_Venue_Country.value, dropdown_Venue_City.value, change.new, dropdown_Genre_Home.value, dropdown_Home_Name.value, bounded_date.value)
def dropdown_Genre_Home_eventhandler(change):
    common_filtering(dropdown_Venue_Country.value, dropdown_Venue_City.value, dropdown_Segment_Home.value, change.new, dropdown_Home_Name.value, bounded_date.value)
def dropdown_Home_Name_eventhandler(change):
    common_filtering(dropdown_Venue_Country.value, dropdown_Venue_City.value, dropdown_Segment_Home.value, dropdown_Genre_Home.value, change.new, bounded_date.value)
def bounded_date_eventhandler(change):
    common_filtering(dropdown_Venue_Country.value, dropdown_Venue_City.value, dropdown_Segment_Home.value, dropdown_Genre_Home.value, dropdown_Home_Name.value, change.new)
    

    
dropdown_Venue_Country.observe(dropdown_Venue_Country_eventhandler, names='value')
dropdown_Venue_City.observe(dropdown_Venue_City_eventhandler, names='value')
dropdown_Segment_Home.observe(dropdown_Segment_Home_eventhandler, names='value')
dropdown_Genre_Home.observe(dropdown_Genre_Home_eventhandler, names='value')
dropdown_Home_Name.observe(dropdown_Home_Name_eventhandler, names='value')
bounded_date.observe(bounded_date_eventhandler, names='value')

In [7]:
input_widgets = widgets.HBox([dropdown_Venue_Country, dropdown_Venue_City, dropdown_Segment_Home,dropdown_Genre_Home,dropdown_Home_Name,bounded_date])

In [8]:
tab = widgets.Tab([output, plot_output,plot_output2,plot_output3])
tab.set_title(0, 'Exploration')
tab.set_title(1, 'Min_Price')
tab.set_title(2, 'Avg_Price')
tab.set_title(3, 'Max_Price')


In [9]:
dashboard = widgets.VBox([tab])
display(dropdown_Venue_Country)
display(dropdown_Venue_City)
display(dropdown_Segment_Home)
display(dropdown_Genre_Home)
display(dropdown_Home_Name)
display(bounded_date)
display(dashboard)

Dropdown(description='Country:', options=('ALL', 'Australia', 'Canada', 'Great Britain', 'Ireland', 'New Zeala…

Dropdown(description='City:', options=('ALL', 'Abbotsford', 'Aberdeen', 'Aberystwyth', 'Adelaide ', 'Agoura Hi…

Dropdown(description='Segment:', options=('ALL', 'Arts & Theatre', 'Film', 'Miscellaneous', 'Music', 'Sports')…

Dropdown(description='Genre:', options=('ALL', 'Action/Adventure', 'Alternative', 'Animation', 'Ballads/Romant…

Dropdown(description='Home:', options=('ALL', '$UICIDEBOY$', "'Twas the Night Before... by Cirque du Soleil", …

DatePicker(value=None, description='Pick a Date')